In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ModuleNotFoundError: No module named 'sklearn'

In [7]:
with open('pickles/train_dfs.pkl', 'rb') as archivo:
    train_dfs = pickle.load(archivo)
with open('pickles/test_dfs.pkl', 'rb') as archivo:
    test_dfs = pickle.load(archivo)

In [9]:
len(train_dfs), len(test_dfs)

(9, 19)

In [10]:
train_dfs[0].shape, train_dfs[0]

((16598, 16),
 array([[0.8442645603171938, 0.4227922711060506, 0.4646260990921964, ...,
         False, False, False],
        [0.8393287286758949, 0.3996053454651987, 0.46635100193025864, ...,
         False, False, False],
        [0.8393287286758949, 0.3926985806153541, 0.46630306401273486, ...,
         False, False, False],
        ...,
        [0.7071273463082389, 0.621114949084335, 0.7566218751140601, ...,
         False, False, False],
        [0.7038104657470432, 0.6277750462251837, 0.7544186660517622, ...,
         False, False, False],
        [0.6988351499595439, 0.6253083231365847, 0.7553296589232483, ...,
         False, False, False]], dtype=object))

In [ ]:

# Convierte los datos de series temporales en un formato adecuado para LSTM
data = data.dropna()
data = data.reset_index(drop=True)
data['time'] = data['time'].astype('int64')

# Crea el modelo LSTM
model = Sequential()
model.add(LSTM(100, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

# Entrena el modelo LSTM
model.fit(train_data['value'].values.reshape(-1, 1), train_data['time'].values, epochs=100, batch_size=10)

# Predice la primera variable de los datos de series temporales
predictions = model.predict(test_data['value'].values.reshape(-1, 1))

# Grafica los resultados
plt.plot(test_data['time'], test_data['value'], label='Valor real')
plt.plot(test_data['time'], predictions, label='Predicción')
plt.legend()
plt.show()